# FedLib: Simulating FedAvg using FedLib virtual Federated environment

## Importing supportive libaries
This notebook shows a demo on PyTorch back-end model impelementation.

In the very begining, we import the supporting libraries.

In [ ]:
import torch
import numpy as np
import copy

from fedlib.ve import BaseSimulator as Simulator
from fedlib.lib import Server, Client
from fedlib.networks import resnet20
from fedlib.lib.sampler import random_sampler
from fedlib.lib.algo.fedavg import Trainer 
from fedlib.datasets import partition_data, get_dataloader,get_client_dataloader


## Define arguments
Here we define arguments. To show an intuitive example, we show the demo store all the parameters in a dictionary in the following code block.
We also provide APIs for you create your arguments in a `*.yaml` file.

In [ ]:
args = {}
args["n_clients"] = 100
args["device"] = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
args['sample_fn'] = random_sampler
args['trainer'] = Trainer()
args['communicator'] = None
args["test_dataset"] = None
args["partition"] = "noniid-labeldir"
args["dataset"] = "cifar10"
args["datadir"] = "./data"
args["beta"] = 0.5
args["batch_size"] = 64
args["global_model"] = resnet20()
args["lr"] = 0.01
args["optimizer"] = "SGD"
args["lr_scheduler"] = "ExponentialLR"

Load test dataset for server, and passing it as an argument

In [ ]:
X_train, y_train, X_test, y_test, net_dataidx_map, traindata_cls_counts = partition_data(
    args["dataset"], args["datadir"], args['partition'], args['n_clients'], beta=args['beta'])
n_classes = len(np.unique(y_train))
train_dl_global, test_dl_global, train_ds_global, test_ds_global = get_dataloader(args["dataset"],
                                                                                    args["datadir"],
                                                                                      args["batch_size"],
                                                                                      32)
args["test_dataset"] = test_dl_global

## Create server and clients objects
Here we use the arguments we defined before, and create server and clients.

In [ ]:
server = Server(**args)
clients = {}

data_loaders = get_client_dataloader(args["dataset"], args["datadir"], args['batch_size'], 32, net_dataidx_map)


for id in range(args["n_clients"]):
    # dataidxs = net_dataidx_map[id]
    args["id"] = id
    # args["trainloader"], _, _, _ = get_dataloader(args["dataset"], args["datadir"], args['batch_size'], 32, dataidxs)
    args["trainloader"] = data_loaders[id]
    args["model"] = copy.deepcopy(args["global_model"])
    clients[id] = Client(**args)

criterion = torch.nn.CrossEntropyLoss()


## Create simulator

Simulator simulates the virtual federated learning environments, and run server and clients on single device.

In [ ]:
simulator = Simulator(server=server, clients=clients, communication_rounds=10,n_clients= 100,sample_rate=.1)

## Run simulator
User API Simulator.run

In [ ]:
simulator.run(local_epochs=2)